# Building ANCOR Dataset

In [ ]:
from french_crs.ancor2dataset import dataset_builder

# Do-able with two nested loops
# for method in ["balanced", "representative", "window"]:
#     for subcorpus in [["INDIRECTE"],["DIRECTE"],["ANAPHORE"]]:

dataset_params={
    # There are other three possible strategies: (1)"balanced", (2)"representative" and (3)"window"
    "strategy" : "balanced,
    "ancor_corpus_path" : "../DISTRIB_ANCOR/",
    # All possibilities are: ["corpus_OTG","corpus_UBS","corpus_ESLO" ,"corpus_ESLO_CO2"]
    "sub_corpus_filter" : ["corpus_OTG","corpus_UBS","corpus_ESLO" ,"corpus_ESLO_CO2"], 
    # All possibilities are: ["DIRECTE", "INDIRECTE", "ANAPHORE"]
    "coreference_type_filter" : ["DIRECTE"], 
    "dataset_output_folder" : "../datasets/",
    # This will be used only in window strategy
    "window_size" : 5 
}

dataset = dataset_builder(**dataset_params)
dataset.build_dataset(file_analysis_alert=True)
dataset.merge_dataset(delete_original_after_merge=False)

In [3]:
import glob

json_relations_lined_up={}
counter=0

for sub_corpus in ["corpus_OTG","corpus_UBS","corpus_ESLO","corpus_ESLO_CO2"]:

    ancor_subcorpus_path = "../DISTRIB_ANCOR/"+sub_corpus+'/'

    corpus_files = [file[file.rfind(
        "/")+1:-4] for file in glob.glob(ancor_subcorpus_path+"annotation_integree/*.xml")]

    list_dataframes = []

    for subfile in corpus_files:

        root_xml,root_aa,data_source=set_file_config(sub_corpus, subfile)
        json_mentions=generate_json_mentions(root_xml,root_aa)
        json_mentions=remove_empty_json_mentions(json_mentions)
        json_relations=generate_json_relations()
        if json_relations == {}:
            continue
        json_coreference_chains=generate_json_chains()
        
        print("starting",subfile,"at",sub_corpus)
        
        for pair_id in json_relations:            

            if json_relations[pair_id]["TYPE"] in ["DIRECTE", "INDIRECTE", "ANAPHORE"]:
                
                counter+=1
                coref_features=generate_coref_relations_json_features(pair_id,counter,json_relations,
                                                                      json_mentions,data_source,
                                                                      is_positive_instance=True)
                json_relations_lined_up[counter]=coref_features

            
                list_mentions_sorted_seperated=[]
                df_mentions = pd.DataFrame.from_dict(json_mentions, orient='index')
                df_mentions = df_mentions.sort_values(by=['START_ID'])
                list_mentions_sorted = list(df_mentions.index)
                
                relation_left=json_relations[pair_id]["LEFT_UNIT"]["ID"]
                relation_right=json_relations[pair_id]["RIGHT_UNIT"]["ID"]
                
                if json_mentions[relation_left]["START_ID"] > json_mentions[relation_right]["START_ID"]:
                    relation_left,relation_right=relation_right,relation_left

                start_index=list_mentions_sorted.index(relation_left)+1
                end_index=list_mentions_sorted.index(relation_right)

                mentions_in_between=list_mentions_sorted[start_index:end_index]

                result="Nothing"
                for ment in reversed(mentions_in_between):
                    for index, chain in json_coreference_chains.items():
                        if not (ment in json_coreference_chains[index] and  relation_right in json_coreference_chains[index]): 
                                result=(ment,relation_right)
                                break
                    else:
                        continue

                    break
                    
                if result!="Nothing":
                    
                    counter+=1
                    coref_features=generate_coref_relations_json_features(-1,counter,json_relations,
                                                                          json_mentions,data_source,
                                                                          is_positive_instance=False,
                                                                          pair_left_id=result[0],
                                                                          pair_right_id=result[1])
                    json_relations_lined_up[counter]=coref_features
                    
                    

            if json_relations[pair_id]["TYPE"] in ["ASSOC", "ASSOC_PRONOM"]:

                counter+=1
                coref_features=generate_coref_relations_json_features(pair_id,counter,json_relations,
                                                                      json_mentions,data_source,
                                                                      is_positive_instance=True,
                                                                      is_assoc=True)
                json_relations_lined_up[counter]=coref_features


df_coref_relations = pd.DataFrame.from_dict(json_relations_lined_up, orient='index')
df_coref_relations.to_excel("Dataset_ANCOR_Balanced.xlsx")

df_coref_relations

NameError: name 'set_file_config' is not defined

In [ ]:
def generate_coref_relations_json_features(pair_id,counter_id,json_relations,json_mentions,
                                           data_source,is_positive_instance=True,
                                           pair_left_id=None,pair_right_id=None,is_assoc=False):

    if is_positive_instance:
        relation_type=json_relations[pair_id]["TYPE"]
        relation_left=json_relations[pair_id]["LEFT_UNIT"]
        relation_right=json_relations[pair_id]["RIGHT_UNIT"]
        
        if relation_left["START_ID"] > relation_right["START_ID"]:
            relation_left,relation_right=relation_right,relation_left
        
        MENTION_LEFT_ID=relation_left["ID"]
        MENTION_RIGHT_ID=relation_right["ID"]
        
        if is_assoc:
            is_positive_instance=False
            
        
    else:
        relation_type="NON_COREFERENCE"
        relation_left=json_mentions[pair_left_id]
        relation_right=json_mentions[pair_right_id]
        
        if relation_left["START_ID"] > relation_right["START_ID"]:
            relation_left,relation_right=relation_right,relation_left
        
        MENTION_LEFT_ID=pair_left_id
        MENTION_RIGHT_ID=pair_right_id




    string_text = data_source[relation_left["END_ID"]:relation_right["START_ID"]]
    string_text=re.sub("<[a-zA-Z0-9\s=\",/.]+>", " ", string_text).strip().split()
    DISTANCE_MENTION = int(relation_right["NUM"])-int(relation_left["NUM"])-1
    DISTANCE_WORD = len(string_text)
    DISTANCE_CHAR = len("".join(string_text))


    tf2yn = {True: "YES", False: "NO"}

    left_content = str(relation_left["CONTENT"])
    right_content = str(relation_right["CONTENT"])
    left_content_list = left_content.lower().split()
    right_content_list = right_content.lower().split()
    len_of_words_min = min(len(left_content_list), len(right_content_list))
    len_of_words_max = max(len(left_content_list), len(right_content_list))
    len_intersection = len(
        [value for value in left_content_list if value in right_content_list])


    ID_FORM = tf2yn[relation_left["CONTENT"]==relation_right["CONTENT"]]
    ID_SUBFORM = tf2yn[len_intersection>0]

    INCL_RATE = len_intersection/len_of_words_min
    COM_RATE = len_intersection/len_of_words_max

    ID_DEF = tf2yn[relation_left["DEF"]==relation_right["DEF"]]
    if relation_left["DEF"] == "UNK" or relation_right["DEF"] == "UNK":
        ID_DEF = "UNK"

    ID_GP = tf2yn[relation_left["GP"]==relation_right["GP"]]
    if relation_left["GP"] == "UNK" or relation_right["GP"] == "UNK":
        ID_GP = "UNK"

    ID_TYPE = tf2yn[relation_left["TYPE"]==relation_right["TYPE"]]
    if relation_left["TYPE"] == "UNK" or relation_right["TYPE"] == "UNK":
        ID_TYPE = "UNK"

    ID_EN = tf2yn[relation_left["EN"]==relation_right["EN"]]
    if relation_left["EN"] == "UNK" or relation_right["EN"] == "UNK":
        ID_EN = "UNK"

    ID_GENDER = tf2yn[relation_left["GENRE"]==relation_right["GENRE"]]
    if relation_left["GENRE"] == "UNK" or relation_right["GENRE"] == "UNK":
        ID_GENDER = "UNK"

    ID_NUMBER = tf2yn[relation_left["NUM"]==relation_right["NUM"]]
    if relation_left["NUM"] == "UNK" or relation_right["NUM"] == "UNK":
        ID_NUMBER = "UNK"

    EMBEDDED = tf2yn[((left_content in right_content) or (right_content in left_content))]

    ID_PREVIOUS = tf2yn[relation_left["PREVIOUS"]==relation_right["PREVIOUS"]]
    if relation_left["PREVIOUS"] == "^" or relation_right["PREVIOUS"] == "^":
        ID_PREVIOUS = "UNK"

    ID_NEXT = tf2yn[relation_left["NEXT"]==relation_right["NEXT"]]
    if relation_left["NEXT"] == "^" or relation_right["NEXT"] == "^":
        ID_NEXT = "UNK"
        
    coref_relation_json_feature={
        
                                "SUB_CORPUS":sub_corpus,
                                "FILE_NAME":subfile,
                                "COREF_TABLE_ID":counter_id,
                                "COREF_TYPE":relation_type,
                                "COREF_ANCOR_ID":pair_id,
                                "MENTION_LEFT_ID":MENTION_LEFT_ID,
                                "MENTION_RIGHT_ID":MENTION_RIGHT_ID,
                                "LEFT_CONTENT":relation_left["CONTENT"],                                     
                                "RIGHT_CONTENT":relation_right["CONTENT"],                                     
                                "LEFT_TYPE":relation_left["TYPE"], 
                                "RIGHT_TYPE":relation_right["TYPE"], 
                                "LEFT_DEF":relation_left["DEF"],
                                "RIGHT_DEF":relation_right["DEF"], 
                                "LEFT_GP":relation_left["GP"],
                                "RIGHT_GP":relation_right["GP"],
                                "LEFT_GENRE":relation_left["GENRE"],
                                "RIGHT_GENRE":relation_right["GENRE"],
                                "LEFT_NB":relation_left["NB"],
                                "RIGHT_NB":relation_right["NB"],
                                "LEFT_EN":relation_left["EN"],
                                "RIGHT_EN":relation_right["EN"],
                                "ID_FORM": ID_FORM, 
                                "ID_SUBFORM": ID_SUBFORM, 
                                "INCL_RATE": INCL_RATE, 
                                "COM_RATE": COM_RATE, 
                                "ID_DEF": ID_DEF, 
                                "ID_GP": ID_GP, 
                                "ID_TYPE": ID_TYPE,
                                "ID_EN": ID_EN, 
                                "ID_GENDER": ID_GENDER, 
                                "ID_NUMBER": ID_NUMBER, 
                                "DISTANCE_MENTION": DISTANCE_MENTION, 
                                "DISTANCE_WORD": DISTANCE_WORD,
                                "DISTANCE_CHAR": DISTANCE_CHAR, 
                                "EMBEDDED": EMBEDDED, 
                                "ID_PREVIOUS": ID_PREVIOUS, 
                                "ID_NEXT": ID_NEXT, 
                                "IS_CO_REF": int(is_positive_instance)
    }
    
    return(coref_relation_json_feature)
                    

In [ ]:
import xml.etree.ElementTree as ET
import json
import pandas as pd
import re
import random
import glob
import networkx as nx
import os


def set_file_config(sub_corpus, file):

    path_file = "../DISTRIB_ANCOR/"+sub_corpus+'/'

    tree_xml = ET.parse(
        path_file+'annotation_integree/'+file+'.xml')
    tree_aa = ET.parse(
        path_file+'aa_fichiers/'+file+'.aa')
    data_source = open(
        path_file+'ac_fichiers/'+file + '.ac', "r").read()

    root_xml = tree_xml.getroot()
    root_aa = tree_aa.getroot()
    
    return(root_xml,root_aa,data_source)
    

# return mentions in json format
def generate_json_mentions(root_xml,root_aa):

    json_mentions = {}

    for unit in root_xml.iter("unit"):
        data_character = {}
        data_character["TYPE"] = unit.find("./characterisation/type").text

        for anchor in root_xml.iter("anchor"):
            if anchor.attrib["id"] == unit.attrib["id"]:
                data_character["NUM"] = anchor.attrib["num"]

        for feat in unit.findall("./characterisation/featureSet/feature"):
            data_character[feat.attrib["name"]] = feat.text

        for unit_aa in root_aa.iter("unit"):
            if unit_aa.attrib["id"] == unit.attrib["id"]:
                data_character["START_ID"] = int(unit_aa.find(
                    "./positioning/start/singlePosition").attrib["index"])
                data_character["END_ID"] = int(unit_aa.find(
                    "./positioning/end/singlePosition").attrib["index"])

        json_mentions[unit.attrib["id"]] = data_character

    return(json_mentions)

# return mentions in json format and eliminates all the features that are not presented.
def remove_empty_json_mentions(json_mentions,features_to_check=["CONTENT", "PREVIOUS"]):

    json_mentions_= json_mentions.copy()
    for mention in list(json_mentions_):
        for feature in features_to_check:
            if feature not in json_mentions_[mention]:
                del json_mentions_[mention]
                break

    return(json_mentions_)


# return relations in json format based on the json mentions
def generate_json_relations():

    json_relations={}

    for relation in root_aa.iter("relation"):
        data_character = {}
        data_character["TYPE"] = relation.find("./characterisation/type").text

        for feat in relation.findall("./characterisation/featureSet/feature"):
            data_character[feat.attrib["name"]] = feat.text

        unit_ids = []
        for feat in relation.findall("./positioning/term"):
            unit_ids.append(feat.attrib["id"])

        if (unit_ids[0] in json_mentions.keys()) and (unit_ids[1] in json_mentions.keys()):
            if (len(unit_ids) == 2):
                data_character["LEFT_UNIT"] = {
                    "ID": unit_ids[0], **json_mentions[unit_ids[0]]}
                data_character["RIGHT_UNIT"] = {
                    "ID": unit_ids[1], **json_mentions[unit_ids[1]]}

            json_relations[relation.attrib["id"]] = data_character

    return(json_relations)


# return chains in json format based on the json mentions and relations.
def generate_json_chains(coreference_type=["DIRECTE", "INDIRECTE", "ANAPHORE"]):
    G = nx.Graph()
    chain_id = 0
    json_coreference_chains = {}

    for chain in json_relations:
        if json_relations[chain]["TYPE"].strip() in coreference_type:
            chain_left = json_relations[chain]["LEFT_UNIT"]["ID"]
            chain_right = json_relations[chain]["RIGHT_UNIT"]["ID"]
            G.add_edge(chain_left, chain_right,
                       coref_type=json_relations[chain]["TYPE"].strip())

    for subgraph in list(nx.connected_components(G)):
        json_coreference_chains[chain_id] = []
        for node in subgraph:
            json_coreference_chains[chain_id].append(node)
        chain_id += 1

    return(json_coreference_chains)

# Dataset Splitting

In [ ]:
# Two ways of Training: (1) commented codes down with parameter.
#                       (2) uncommented with mentioned explicity the config file.

from french_crs.fast_model_training import dataset_splitter

# ds1=dataset_splitter("./Dataset_ANCOR_Balanced.xlsx",
#                     "./Dataset_ANCOR_Balanced_Train.xlsx",
#                     "./Dataset_ANCOR_Balanced_Test.xlsx",
#                       split_config_json="./split_config.json")

# ds1.dataset_splitter_by_file(lower_rate=0.20,upper_rate=0.50, files_num=68)


ds2=dataset_splitter("./Dataset_ANCOR_Balanced.xlsx",
                    "./Dataset_ANCOR_Balanced_Train.xlsx",
                    "./Dataset_ANCOR_Balanced_Test.xlsx",
                      split_config_json="./split_config.json")

dict_files=ds2.dataset_splitter_by_json_config()


# Model Training

In [1]:
from french_crs.fast_model_training import model_trainer

"""
12 possibilities for "train" and "test" variables

balanced_ANAPHORE
balanced_DIRECTE
balanced_Full
balanced_INDIRECTE
representative_ANAPHORE
representative_DIRECTE
representative_Full
representative_INDIRECTE
window_ANAPHORE
window_DIRECTE
window_Full
window_INDIRECTE
"""

train="balanced_Full"
test="balanced_Full_Soon"

model=model_trainer( "../datasets/"+ train+"/"+train+"_Train.xlsx",
                     "../datasets/"+ test+"/"+test+"_Test.xlsx",
                     "../datasets/"+ train+"/"+train+"_Test_Pred.xlsx",
                     "IS_CO_REF",
                     "IS_CO_REF"
                   )

model.columns_drop_list = ["SUB_CORPUS",
                            "FILE_NAME",
                            "COREF_TABLE_ID",
                            "COREF_TYPE",
                            "COREF_ANCOR_ID",
                            "MENTION_LEFT_ID",
                            "MENTION_RIGHT_ID",
                            "LEFT_CONTENT",
                            "RIGHT_CONTENT",
                            "LEFT_DEF",
                            "RIGHT_DEF",
                            "ID_DEF",
                            "DISTANCE_MENTION",
                            "DISTANCE_WORD",
                            "DISTANCE_CHAR"]

model.convert_columns_to_numeric()
print("Model_Name :","Model_ANCOR_"+train+".model\n")
print("Train Dataset :",train+"_Train.xlsx")
print("Test  Dataset :",test+"_Test.xlsx")
performance=model.train_model_random_forest(model_name="../pre-trained language models/Model_ANCOR_"+train+".model",max_depth=10, random_state=0,n_estimators=250)
performance

Model_Name : Model_ANCOR_balanced_Full_Soon.model

Train Dataset : balanced_Full_Soon_Train.xlsx
Test  Dataset : balanced_Full_Soon_Test.xlsx


{'precision_score_pos': 0.8380271822894774,
 'recall_score_pos': 0.8590922148807814,
 'f1_score_pos': 0.8484289665933754,
 'precision_score_neg': 0.8545373665480427,
 'recall_score_neg': 0.8329238329238329,
 'f1_score_neg': 0.8435921832686818}

In [14]:
train="balanced_Full"
test="window_Full"

# SCORCH Chains Building

In [15]:
from french_crs.pairs2chains import chains_builder

model_chains=chains_builder(path_gold_file="../datasets/"+ train+"/"+train+"_Test_Pred.xlsx", 
                            path_model_file="../datasets/"+ train+"/"+train+"_Test_Pred.xlsx",
                            gold_column="IS_CO_REF",
                            model_column="Prediction",
                            scorch_output_path="../",
                            threshold=0.5)

model_chains.generate_gold_model_json_output(mode="train")

SCORCH json formats saved at following addresses:
/Users/mehdi.mirzapour/French-CRS/coref_chains_gold.json
/Users/mehdi.mirzapour/French-CRS/coref_chains_pred.json


# SCORCH Outcome

In [11]:
import os
bashCommand = "scorch ../coref_chains_gold_b.json ../coref_chains_pred_r.json > ../mm.txt"
os.system(bashCommand)
f = open("../mm.txt",'r')
message = f.read()
print(message)

# Putting All Chains Together

In [ ]:
import docx
import os
from french_crs.model_training import model_trainer
from french_crs.pairs2chains import chains_builder

mydoc = docx.Document()

style = mydoc.styles['Normal']
font = style.font
font.name = 'MS Gothic'
font.size = docx.shared.Pt(10)


train_test_list=[
    "balanced_ANAPHORE",
    "balanced_DIRECTE",
    "balanced_Full",
    "balanced_INDIRECTE",
    "representative_ANAPHORE",
    "representative_DIRECTE",
    "representative_Full",
    "representative_INDIRECTE",
    "window_ANAPHORE",
    "window_DIRECTE",
    "window_Full",
    "window_INDIRECTE"
    ]

# train_test_list=[
#     ["balanced_ANAPHORE","window_ANAPHORE"],
#     ["balanced_DIRECTE","window_DIRECTE"],
#     ["balanced_Full","window_Full"],
#     ["balanced_INDIRECTE","window_INDIRECTE"],
#     ["representative_ANAPHORE","window_ANAPHORE"],
#     ["representative_DIRECTE","window_DIRECTE"],
#     ["representative_Full","window_Full"],
#     ["representative_INDIRECTE","window_INDIRECTE"],
#     ["window_ANAPHORE","window_ANAPHORE"],
#     ["window_DIRECTE","window_DIRECTE"],
#     ["window_Full","window_Full"],
#     ["window_INDIRECTE","window_INDIRECTE"]
#     ]

counter=0

# for train in train_test_list:
#     for test in train_test_list:

for train in train_test_list:
    
    test=train
    
    counter+=1

    model=model_trainer( "../datasets/"+ train+"/"+train+"_Train.xlsx",
                 "../datasets/"+ test+"/"+test+"_Test.xlsx",
                 "../datasets/"+ train+"/"+train+"_Test_Pred.xlsx",
                 "IS_CO_REF",
                 "IS_CO_REF"
               )

    model.columns_drop_list = ['m1_DEF', 'm2_DEF', 'ID_DEF',
                               'DISTANCE_MENTION','DISTANCE_WORD',
                               'DISTANCE_CHAR']

    model.convert_columns_to_numeric()
    performance=model.train_model_random_forest(model_name="../pre-trained language models/Model_ANCOR_"+train+".model",max_depth=10, random_state=0,n_estimators=250)


    model_chains=chains_builder(path_gold_file="../datasets/"+ train+"/"+train+"_Test_Pred.xlsx", 
                        path_model_file="../datasets/"+ train+"/"+train+"_Test_Pred.xlsx",
                        gold_column="IS_CO_REF",
                        model_column="Prediction",
                        scorch_output_path="../",
                        threshold=0.5)

    model_chains.generate_gold_model_json_output(mode="train")


    bashCommand = "scorch ../coref_chains_gold.json ../coref_chains_pred.json > ../output.txt"
    os.system(bashCommand)
    f = open("../output.txt",'r')
    message = f.read()
    f.close()


    mydoc.add_paragraph("Model_Name : "+"Model_ANCOR_"+train+".model")
    mydoc.add_paragraph("Train Dataset : "+train+"_Train.xlsx")
    mydoc.add_paragraph("Test  Dataset : "+test+"_Test.xlsx")
    mydoc.add_paragraph("\n")
    mydoc.add_paragraph(str(performance))
    mydoc.add_paragraph("\n")
    mydoc.add_paragraph(message)
    mydoc.add_page_break()

    print(counter)


mydoc.save("../pre-trained language models/Performance Analysis.docx")

#  Model Testing

In [ ]:
from model_training import model_tester

model_parameter={
                "model_name" : "./Models/Random_Forest_(Normal)_OTG_Neg_90_Pos_10.model",
                "input_file" : "./Datasets/corpus_ALL_Window_30_Test.xlsx",
                "output_file" : "./Datasets/corpus_ALL_Window_30_Test_Called_Seperately.xlsx",
                "column_gold" : "IS_CO_REF",
                "column_outcome" : "Prediction",
                "threshold" : 0.5
                }

model=model_tester(**model_parameter)
model.apply_model_to_dataset()